## Image Classification

In [2]:
import cv2
import numpy as np
import random
import tensorflow as tf

In [3]:
# Draw random shape (circle, square, or triangle)
def draw_shape(canvas):
    shape_type = random.choice(['circle', 'square', 'triangle'])
    color=tuple(np.random.randint(0,256,3).tolist())
    if shape_type == 'circle':
        center = (np.random.randint(20,80), np.random.randint(20,80))
        radius = np.random.randint(5, 15)
        cv2.circle(canvas, center, radius, color, -1)
    elif shape_type == 'square':
        start_point = (np.random.randint(10,60), np.random.randint(10, 60))
        end_point = (start_point[0]+ np.random.randint(10,20), start_point[1]+np.random.randint(10,20))
        cv2.rectangle(canvas, start_point, end_point, color, -1)
    elif shape_type == 'triangle':
        pt1 = (np.random.randint(10,90), np.random.randint(10,90))
        pt2 = (np.random.randint(10,90), np.random.randint(10,90))
        pt3 = (np.random.randint(10,90), np.random.randint(10,90))
        points = np.array([pt1, pt2, pt3], np.int32)
        points = points.reshape((-1, 1, 2))
        cv2.polylines(canvas, [points], isClosed=True, color=color, thickness=1)
        cv2.fillPoly(canvas, [points], color)
    return shape_type, canvas

In [4]:
# Generate data set
def generate_dataset(num_samples):
    images = []
    labels = []
    for _ in range(num_samples):
        shape_type, canvas = draw_shape(np.zeros((100, 100, 3), dtype=np.uint8))
        images.append(canvas)
        if shape_type == 'circle':
            labels.append(0)
        elif shape_type == 'square':
            labels.append(1)
        elif shape_type == 'triangle':
            labels.append(2)
    return np.array(images), np.array(labels)

In [5]:
# Generate 1000 samples
num_samples = 1000
X, y = generate_dataset(num_samples)
print(len(X), len(y), num_samples)

1000 1000 1000


In [6]:
# Shuffle dataset
indices = np.arange(num_samples)
np.random.shuffle(indices)
X = X[indices]
y = y[indices]

In [7]:
# Split data set for training and tests
split_ratio = 0.8
split_index = int(split_ratio * num_samples)
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

In [8]:
# Convert labels to one-hot encoding
num_classes = 3
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [9]:
# Define CNN Architectur
model = tf.keras.Sequential([
   tf.keras.Input(shape=(100, 100, 3)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

2025-03-13 12:08:40.844927: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2025-03-13 12:08:40.844988: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 36.00 GB
2025-03-13 12:08:40.844998: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 13.50 GB
2025-03-13 12:08:40.845028: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-03-13 12:08:40.845044: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [10]:
# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
# Train model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10


2025-03-13 12:08:41.265368: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - accuracy: 0.3995 - loss: 4.8551 - val_accuracy: 0.5250 - val_loss: 0.9372
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.7014 - loss: 0.7543 - val_accuracy: 0.5800 - val_loss: 0.8233
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.8736 - loss: 0.4129 - val_accuracy: 0.6850 - val_loss: 0.7235
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.9642 - loss: 0.1583 - val_accuracy: 0.7100 - val_loss: 0.7432
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.9959 - loss: 0.0503 - val_accuracy: 0.7350 - val_loss: 0.7007
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 1.0000 - loss: 0.0215 - val_accuracy: 0.7700 - val_loss: 0.7227
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 1.0000 - loss: 0.0098 - val_accuracy: 0.7700 - val_loss: 0.7873
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 1.0000 - loss: 0.0057 - val_accuracy: 0.7800 - val_loss: 0.

In [12]:
# Evaluate model
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print('\nTest accuracy:', test_acc)

7/7 - 0s - 15ms/step - accuracy: 0.7750 - loss: 0.8721

Test accuracy: 0.7749999761581421
